# Unit 3: Detect and localise person

<img src="img/summit_xl_instruction_manual_unit3_intro.png"/>

<img src="img/robotnik_logo.png" width="300"/>

<img src="img/robotignite_logo_text.png" width="300"/>

In this unit you are goint to learn how to detect a person using two different systems:<br>
* **Detect through leg detection with laser sensor**: This system detect persons based on laser readings of leg patterns. This allows you to have even the position of the person, not only detect if there is a person or not.
* **Detect person through a PTZ RGB normal camera**: This system doesnt give you directly the position of the person, but detects the persons through Video with OpenCV.
* As extra, in this unit you will have to learn how to move the PTZ camera around.

## Person leg Detector

This is a classical way to detect people. Its fast and relatively robust. You will use a ROS package called **leg_detector**, for more information http://wiki.ros.org/leg_detector.<br>

The first step is to generate a launch file that sets all the parameters:

**start_legdetector.launch**

In [ ]:
<launch> 
    <arg name="scan" default="/hokuyo_base/scan" />
    <arg name="machine" default="localhost" />
    <arg name="user" default="" />
    <!-- Leg Detector -->
    <node pkg="leg_detector" type="leg_detector" name="leg_detector" args="scan:=$(arg scan) $(find leg_detector)/config/trained_leg_detector.yaml" respawn="true" output="screen">
        <param name="fixed_frame" type="string" value="map" />
    </node>
    
</launch> 

As you can see here the only thing that you have to set really is the **scan** topic, which is the topic for the laser. The rest of parameters are set in a yaml file **trained_leg_detector.yaml**, which you can leave as it is unless you want to optimise or make set all the paremeters for finer tuning.

Once you have it, you should be able to launch it and start detecting people right away. The detections will be published in the topic **/leg_tracker_measurements**. This topic is of type **PositionMeasurementArray**

In [ ]:
### PositionMeasurementArray

std_msgs/Header header                                                                                                         
  uint32 seq                                                                                                                   
  time stamp                                                                                                                   
  string frame_id                                                                                                              
people_msgs/PositionMeasurement[] people                                                                                       
  std_msgs/Header header                                                                                                       
    uint32 seq                                                                                                                 
    time stamp                                                                                                                 
    string frame_id                                                                                                            
  string name                                                                                                                  
  string object_id                                                                                                             
  geometry_msgs/Point pos                                                                                                      
    float64 x                                                                                                                  
    float64 y                                                                                                                  
    float64 z                                                                                                                  
  float64 reliability                                                                                                          
  float64[9] covariance                                                                                                        
  byte initialization                                                                                                          
float32[] cooccurrence

Here you have **Position information** and also **reliability** of the detections.

But lets go a step further. You need to visualise those detections in space. For that by default the leg_detector already has a topic that publishes RVIZ markers called **/visualization_marker**. This is depicts the person detection through a sphere based on the reliability changing its colour and appearance.

<img src="img/summit_xl_instruction_manual_unit3_legmarker0.png" width="800"></img>

<img src="img/summit_xl_instruction_manual_unit3_legmarker1.png" width="800"></img>

You can aslo generate a custom marker that will be much more descriptive when visualised. For that you will have to create a python script that reads the leg_detections from **/leg_tracker_measurements**, and publish it in a new marker_topic with its own custom marker, in this case will be a mesh of the person detected.

**leg_detector_marker_publisher.py**

In [ ]:
#!/usr/bin/env python
import rospy
from people_msgs.msg import PositionMeasurementArray


#!/usr/bin/env python

import rospy
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point


class MarkerBasics(object):

    def __init__(self):
        self.marker_objectlisher = rospy.Publisher('/marker_person_detected_leg', Marker, queue_size=1)
        self.rate = rospy.Rate(1)
        self.init_marker(index=0,z_val=0)
    
    def init_marker(self,index=0, z_val=0):
        self.marker_object = Marker()
        self.marker_object.header.frame_id = "/map"
        self.marker_object.header.stamp    = rospy.get_rostime()
        self.marker_object.ns = "summit_xl"
        self.marker_object.id = index
        self.marker_object.type = Marker.MESH_RESOURCE
        self.marker_object.action = Marker.ADD
        
        my_point = Point()
        self.marker_object.pose.position = my_point
        
        self.marker_object.pose.orientation.x = 0
        self.marker_object.pose.orientation.y = 0
        self.marker_object.pose.orientation.z = 1.0
        self.marker_object.pose.orientation.w = 1.0
        self.marker_object.scale.x = 1.0
        self.marker_object.scale.y = 1.0
        self.marker_object.scale.z = 1.0
    
        self.marker_object.color.r = 0.0
        self.marker_object.color.g = 0.0
        self.marker_object.color.b = 0.0
        # This has to be otherwise it will be transparent
        self.marker_object.color.a = 0.0
        self.marker_object.mesh_resource = "package://person_sim/models/person_standing/meshes/standingv2.dae"
        self.marker_object.mesh_use_embedded_materials = True
        # If we want it for ever, 0, otherwise seconds before desapearing
        self.marker_object.lifetime = rospy.Duration(0)

    def update_position(self,position, reliability):
        
        self.marker_object.pose.position = position
        
        self.marker_objectlisher.publish(self.marker_object)

"""
### PositionMeasurementArray

std_msgs/Header header                                                                                                         
  uint32 seq                                                                                                                   
  time stamp                                                                                                                   
  string frame_id                                                                                                              
people_msgs/PositionMeasurement[] people                                                                                       
  std_msgs/Header header                                                                                                       
    uint32 seq                                                                                                                 
    time stamp                                                                                                                 
    string frame_id                                                                                                            
  string name                                                                                                                  
  string object_id                                                                                                             
  geometry_msgs/Point pos                                                                                                      
    float64 x                                                                                                                  
    float64 y                                                                                                                  
    float64 z                                                                                                                  
  float64 reliability                                                                                                          
  float64[9] covariance                                                                                                        
  byte initialization                                                                                                          
float32[] cooccurrence
"""
class LegDetector(object):
    def __init__(self):
        rospy.Subscriber("/leg_tracker_measurements", PositionMeasurementArray, self.leg_detect_callback)

        self.markerbasics_object = MarkerBasics()

    def leg_detect_callback(self,data):
        for people_object in data.people:
            self.markerbasics_object.update_position(people_object.pos, people_object.reliability)
    
    def start_loop(self):
        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

if __name__ == '__main__':
    rospy.init_node('leg_detections_listener_node', anonymous=True)
    legdetector_object = LegDetector()
    legdetector_object.start_loop()
    

If you need more information about how this is done, we highly reccomend you to do the **RVIZ Markers** course in RobotIgniteAcademy.

Here you basically retrieve the position of the leg detection and publish the marker using the mesh **standingv2.dae** as marker.

Bare in mind that because the leg detector doesnt give you orientation information the marker will have always the same orientation unless you detect the orientation of the person by other menas, such as movement direction.

You should get something similar to this:

<img src="img/summit_xl_instruction_manual_unit3_legmarker4.png" width="800"></img>

<p style="background:#EE9023;color:white;">Exercise U3-1</p>

Based on the leg detection data, get the direction in which the person is moving and change the orientation of the custom marker to better match the real person.

<p style="background:#EE9023;color:white;">Exercise U3-2</p>

## Pedestrian Detector

Now you are going to use the PTZ RGB camera to detect persons. For this you will have to first know how to move the camera.

### Move the PTZ Camera

To move the camera you have to publish in the topic **/axis_camera/ptz_command**, a message of type **robotnik_msgs/ptz**.

In [ ]:
# PAN value                                                                                                                                                  
float32 pan                                                                                                                                                  
# Tilt value                                                                                                                                                 
float32 tilt                                                                                                                                                 
# Zoom value                                                                                                                                                 
float32 zoom                                                                                                                                                 
# Flag for relative  movements                                                                                                                               
bool relative

See that because its a PTZ Camera, you can move the Pan, Tilt and also make Zoom. You also can state of the movements are relative or not.

Here you have an example of how you could move the camera. Bare in mind that the movements are based on speed. This menas that you are sending how fast tin radians per second will the pan, or tilt move. This has  to be sent continuously, otherside the movement will stop.

In [ ]:
#!/usr/bin/env python
import time
import rospy
from math import pi, sin, cos, acos
import random
from robotnik_msgs.msg import ptz
from sensor_msgs.msg import JointState
from geometry_msgs.msg import Twist
"""
Topics To Write on:
type: std_msgs/Float64
/joint_pan_position_controller/command                                                                                                                       
/joint_tilt_position_controller/command


rostopic info /axis_camera/ptz_command                                                                                                      
Type: robotnik_msgs/ptz
# PAN value                                                                                                                                                  
float32 pan                                                                                                                                                  
# Tilt value                                                                                                                                                 
float32 tilt                                                                                                                                                 
# Zoom value                                                                                                                                                 
float32 zoom                                                                                                                                                 
# Flag for relative  movements                                                                                                                               
bool relative
"""

class PTZCamMove(object):

    def __init__(self):
        
        rospy.loginfo("PTZ Cam JointMover Initialising...")
        self.pub_joint_pan_position = rospy.Publisher('/axis_camera/ptz_command',
                                                            ptz,
                                                            queue_size=1)
        

    def init_ptz_pos(self):
        rospy.loginfo("Init Pos Start...")
        ptz_object = ptz()
        ptz_object.pan = pan
        ptz_object.tilt = 0.5
        ptz_object.zoom = zoom
        
        rate = rospy.Rate(5)
        for i in range(10):
            self.move_all_joints(ptz_object.pan, ptz_object.tilt, ptz_object.zoom, ptz_object.relative)
            rate.sleep()

    def move_all_joints(self, pan, tilt, zoom, relative=True):
        ptz_object = ptz()
        ptz_object.pan = pan
        ptz_object.tilt = tilt
        ptz_object.zoom = zoom
        ptz_object.relative = relative
        
        self.pub_joint_pan_position.publish(ptz_object)
    
    
    def pan_camera(self):
        
        rate = rospy.Rate(10)
        pan_speed = 1.0
        tilt_speed = 0.0
        zoom = 0.0
        
        rospy.loginfo("Pan left...")
        for i in range(40):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
        
        rospy.loginfo("Pan right...")
        pan_speed *= -1
        for i in range(80):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
        
        rospy.loginfo("Pan center...")
        pan_speed *= -1
        for i in range(40):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
    def tilt_camera(self):
        
        rate = rospy.Rate(10)
        pan_speed = 0.0
        tilt_speed = 1.0
        zoom = 0.0
        
        rospy.loginfo("Tilt left...")
        for i in range(40):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
        
        rospy.loginfo("Tilt right...")
        tilt_speed *= -1
        for i in range(80):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
        
        rospy.loginfo("Tilt center...")
        tilt_speed *= -1
        for i in range(40):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
    def zoom_camera(self):
        
        rate = rospy.Rate(10)
        pan_speed = 0.0
        tilt_speed = 0.0
        zoom = 100.0
        
        rospy.loginfo("Zoom in...")
        for i in range(40):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
        
        rospy.loginfo("Zoom out...")
        zoom *= -1
        for i in range(80):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()
            
        
        rospy.loginfo("Zoom in...")
        zoom *= -1
        for i in range(40):
            self.move_all_joints(pan_speed, tilt_speed, zoom)
            rate.sleep()

    
    def start_loop(self):
        """
        Executed movements in a random way
        :return:
        """
        rospy.loginfo("Start PTZ Moving Test...")
        
        rospy.spin()



if __name__ == "__main__":
    rospy.init_node('ptz_cam_move_node', anonymous=True, log_level=rospy.INFO)
    jointmover_object = PTZCamMove()
    jointmover_object.pan_camera()
    jointmover_object.tilt_camera()
    jointmover_object.zoom_camera()

<p style="background:#EE9023;color:white;">Exercise U3-2</p>

Create a new movement system that checks on the joint_states of the pan and tilt, so that you can send a position , and it will reach that position sending speed commands.

<p style="background:#EE9023;color:white;">END Exercise U3-2</p>

### Detect pedestrians with OpenCV script

You are going to learn how to acces the camera data in ROS, and convert it to OpenCV format. Then you will process the image data to detect pedestrians and draw a bounding box around them.

For this you will create two new scripts:<br>
* **the main detect person**, shich will retrieve the image data from the Summit XL robot ROS topics and send it to a pedestrian detector script. 

* **The pedestrian detector script**: This script will do the tough work of detecting and drawing the bounding box around the person detected.

**detect_person_camera.py**

In [ ]:
#!/usr/bin/env python

import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from pedestrian_detector import PedestrianDetector
from move_ptz_cam import PTZCamMove

class DetectPersonCam(object):

    def __init__(self):
    
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber("/camera/image_raw",Image,self.camera_callback)
        self.pedestrian_detector = PedestrianDetector()
        self.ptz_cam_mover = PTZCamMove()
        self.init_time = rospy.get_time()
        self.now_pitch = 0
        self.now_roll = 0
        self.process_this_frame = True
        
    def camera_callback(self,data):
        
        try:
            # We select bgr8 because its the OpneCV encoding by default
            cv_image = self.bridge_object.imgmsg_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)

        # To process only half of the images for faster performance
        if self.process_this_frame:
            self.pedestrian_detector.detect_pedestrian(cv_image)
        
        self.process_this_frame = not self.process_this_frame
    
    def move_camera(self,roll,pitch):
        self.ptz_cam_mover.move_all_joints(roll, pitch)


def main():
    rospy.init_node('detect_person_camera_node', anonymous=True)
    
    detect_personcam_object = DetectPersonCam()
    
    try:
        rospy.spin()
    except KeyboardInterrupt:
        print("Shutting down")
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Comment on the following:

In [ ]:
self.image_sub = rospy.Subscriber("/camera/image_raw",Image,self.camera_callback)

Here you susbrcibe to the topic where the Camera images are published.

In [ ]:
cv_image = self.bridge_object.imgmsg_to_cv2(data, desired_encoding="bgr8")

Here you get that image and tranform it into OpenCV BGR8bit format. If you want to know more details, please reffer to the **ROS perception Course** in RobotIgniteAcademy.

In [ ]:
if self.process_this_frame:
    ...
self.process_this_frame = not self.process_this_frame

This condition makes the program procss only half of the images, making it much faster.

In [ ]:
self.pedestrian_detector.detect_pedestrian(cv_image)

This fucntion sends the data to the pedestrian detector.

In [ ]:
#! /usr/bin/env python
"""
USAGE
python detect.py --images images
"""
# import the necessary packages
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2


class PedestrianDetector(object):
    def __init__(self):
        # initialize the HOG descriptor/person detector
        self.hog = cv2.HOGDescriptor()
        self.hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    def detect_pedestrian(self, image):

    	image = imutils.resize(image, width=min(400, image.shape[1]))
    	orig = image.copy()
    
    	# detect people in the image
    	(rects, weights) = self.hog.detectMultiScale(image, winStride=(4, 4),
    		padding=(8, 8), scale=1.05)
    
    	# draw the original bounding boxes
    	"""
    	for (x, y, w, h) in rects:
    		cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
        """
    	# apply non-maxima suppression to the bounding boxes using a
    	# fairly large overlap threshold to try to maintain overlapping
    	# boxes that are still people
    	rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    	pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    
    	# draw the final bounding boxes
    	for (xA, yA, xB, yB) in pick:
    		cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
    
    	# show some information on the number of bounding boxes
    	filename = "LiveCam"
    	print("[INFO] {}: {} original boxes, {} after suppression".format(
    		filename, len(rects), len(pick)))
    
    	# show the output images
    	#cv2.imshow("Before NMS", orig)
    	cv2.imshow("Pdestrian CAM", image)
    	cv2.waitKey(1)

When executing this code you should get something similar to this:

<img src="img/summit_xl_instruction_manual_unit3_pedestriandetector.png" width="800"></img>

<p style="background:#EE9023;color:white;">Exercise U3-2</p>

Create a method that based on the data of the pedestrian detection bounding boxes, publishes the positon relative to the Summit XL. For that you will have to make calibration tests to see the size of the bounding box based on the distance, and TF publications. If you need more information on how to do that, in the course ** ROS TF** and **ROS Perception** you will find it. 

<p style="background:#EE9023;color:white;">END Exercise U3-2</p>

## Congratulations, you have finished the theory of this course, now to the micro project!